In [1]:
# --- User inputs (interactive) ---
# NOTE: For multiple cores, organize your data like this:
#   parent_folder/
#       Core1/
#           file1_0-10mm.csv
#           file2_10-20mm.csv
#       Core2/
#           file1_0-10mm.csv
#           file2_10-20mm.csv

directory   = input("📁 Enter the parent folder path (containing core subfolders or CSVs): ").strip()
plot_title  = input("📊 Enter overall plot title: ").strip()
output_dir  = input("💾 Enter output directory (default '.'): ").strip() or "."
fig_w_per_core = float(input("📏 Figure width per core (inches, default 2): ") or 2)
fig_h       = float(input("📐 Figure height (inches, default 10): ") or 10)

# Wentworth bins (µm) and labels
wentworth_bins   = [0, 4, 63, 2000]
wentworth_labels = ["Clay", "Silt", "Sand"]

print(f"\n✅ Ready! Searching for cores in: {directory}\n")


📁 Enter the parent folder path (containing core subfolders or CSVs):  D:\Vent\grain size\processed
📊 Enter overall plot title:  
💾 Enter output directory (default '.'):  D:\Vent\grain size\processed
📏 Figure width per core (inches, default 2):  
📐 Figure height (inches, default 10):  



✅ Ready! Searching for cores in: D:\Vent\grain size\processed



In [2]:
#Cell 2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

plt.rcParams.update({"font.size": 15})


In [3]:
#Cell 3
def find_cores(parent_dir: str) -> dict:
    """Find all cores. Returns dict {core_name: path_to_core_data}"""
    cores = {}
    
    # Check if parent_dir itself contains CSV files (single core)
    csv_files = [f for f in os.listdir(parent_dir) if f.endswith('.csv')]
    if csv_files:
        cores[os.path.basename(parent_dir)] = parent_dir
        return cores
    
    # Otherwise, look for subdirectories with CSV files (multiple cores)
    for item in os.listdir(parent_dir):
        item_path = os.path.join(parent_dir, item)
        if os.path.isdir(item_path):
            # Check if this subdirectory contains CSV files
            if any(f.endswith('.csv') for f in os.listdir(item_path)):
                cores[item] = item_path
    
    if not cores:
        raise FileNotFoundError(f"No CSV files found in {parent_dir} or its subdirectories")
    
    return cores

def extract_depths(file_stem: str):
    """Extract depth information from filename. Supports both mm and cm units."""
    parts = file_stem.split("_")[-1].split("-")
    try:
        # Parse lower depth
        lower_str = parts[0].strip()
        if "cm" in lower_str:
            lo = float(lower_str.replace("cm", "").strip()) * 10  # Convert cm to mm
        elif "mm" in lower_str:
            lo = float(lower_str.replace("mm", "").strip())
        else:
            lo = float(lower_str)  # Assume mm if no unit
        
        # Parse upper depth
        upper_str = parts[1].strip()
        if "cm" in upper_str:
            hi = float(upper_str.replace("cm", "").strip()) * 10  # Convert cm to mm
        elif "mm" in upper_str:
            hi = float(upper_str.replace("mm", "").strip())
        else:
            hi = float(upper_str)  # Assume mm if no unit
            
        return lo, hi, (lo + hi) / 2
    except (ValueError, IndexError) as e:
        raise ValueError(
            f"Bad name '{file_stem}'. Use '..._<lower>-<upper>mm.csv' or '..._<lower>-<upper>cm.csv'."
        ) from e

In [4]:
#Cell 4
def read_multiple_data(directory: str) -> pd.DataFrame:
    frames = []
    for fn in os.listdir(directory):
        if not fn.endswith(".csv"): 
            continue
        fp = os.path.join(directory, fn)
        df = pd.read_csv(fp)
        lo, hi, mid = extract_depths(os.path.splitext(os.path.basename(fp))[0])
        df["Lower Depth"] = lo
        df["Upper Depth"] = hi
        df["Center Depth"] = mid
        frames.append(df)
    if not frames:
        raise FileNotFoundError(f"No CSVs in {directory}")
    return pd.concat(frames, ignore_index=True)


In [5]:
#Cell 5 - Standard plotting function (without dots)
#def plot_wentworth_scale(data, bins, labels, title, figsize, out_pdf, gap_mm=20, colors=None):
#    depths = np.sort(data["Center Depth"].unique())
#    binned = pd.cut(data["um"], bins=bins, labels=labels, right=False, include_lowest=True, ordered=True)
#    tmp = data.assign(class_=binned).groupby(["Center Depth", "class_"], observed=True)["Histogram (%) q3"].sum().reset_index()
#    wide = tmp.pivot(index="Center Depth", columns="class_", values="Histogram (%) q3").reindex(labels, axis=1).fillna(0)
#    cumu = wide.cumsum(axis=1).to_numpy(); fig, ax = plt.subplots(figsize=figsize)
#    idx = np.arange(len(depths)); gaps = np.where(np.diff(depths) > gap_mm)[0]; segs = np.split(idx, gaps + 1)
#    cols = colors or ["#1f77b4", "#17becf", "#bcbd22"]  # Clay, Silt, Sand
#    for i, lab in enumerate(labels):
#        left = np.zeros(len(depths)) if i == 0 else cumu[:, i-1]; right = cumu[:, i]
#        for j, s in enumerate(segs):
#            ys = depths[s]; yl = left[s]; yr = right[s]
#            ypad = np.r_[ys[0]-1e-6, ys, ys[-1]+1e-6]
#            ax.fill_betweenx(ypad, np.r_[yl[0], yl, yl[-1]], np.r_[yr[0], yr, yr[-1]],
#                             color=cols[i], label=lab if j == 0 else None)
#    ax.set(xlim=(0, 100), ylabel="Core Depth (mm)", xlabel="Grain Size (%)", title=title)
#    ax.invert_yaxis(); ax.legend(loc="lower center", bbox_to_anchor=(0.5, -0.15), ncol=len(labels), frameon=False)
#    fig.savefig(out_pdf, bbox_inches="tight", dpi=300); plt.close(fig); print(f"Plot saved as {out_pdf}")


In [6]:
#Cell 5c - Multi-core plotting function with unified y-axis
def plot_multiple_cores(cores_data: dict, bins, labels, title, fig_w_per_core, fig_h, 
                        out_pdf, gap_mm=20, colors=None, add_dots=False):
    """
    Plot multiple cores side-by-side with unified y-axis scale.
    
    Parameters:
    cores_data: dict of {core_name: dataframe}
    add_dots: if True, add dots at measurement points on the Silt peak
    """
    n_cores = len(cores_data)
    cols = colors or ["#653700", "#808080", "#E6DAA6"]  # Clay, Silt, Sand
    
    # Find global min/max depth across all cores for unified scale
    all_depths = []
    for data in cores_data.values():
        all_depths.extend(data["Center Depth"].unique())
    global_min = min(all_depths)
    global_max = max(all_depths)
    
    # Create subplots
    fig, axes = plt.subplots(1, n_cores, figsize=(fig_w_per_core * n_cores, fig_h), 
                             sharey=True, squeeze=False)
    axes = axes.flatten()
    
    # Plot each core
    for idx, (core_name, data) in enumerate(sorted(cores_data.items())):
        ax = axes[idx]
        depths = np.sort(data["Center Depth"].unique())
        
        # Process data
        binned = pd.cut(data["um"], bins=bins, labels=labels, right=False, include_lowest=True, ordered=True)
        tmp = data.assign(class_=binned).groupby(["Center Depth", "class_"], observed=True)["percent"].sum().reset_index()
        wide = tmp.pivot(index="Center Depth", columns="class_", values="percent").reindex(labels, axis=1).fillna(0)
        cumu = wide.cumsum(axis=1).to_numpy()
        
        # Find gaps
        depth_idx = np.arange(len(depths))
        gaps = np.where(np.diff(depths) > gap_mm)[0]
        segs = np.split(depth_idx, gaps + 1)
        
        # Plot each grain size class
        for i, lab in enumerate(labels):
            left = np.zeros(len(depths)) if i == 0 else cumu[:, i-1]
            right = cumu[:, i]
            
            for j, s in enumerate(segs):
                ys = depths[s]
                yl = left[s]
                yr = right[s]
                ypad = np.r_[ys[0]-1e-6, ys, ys[-1]+1e-6]
                ax.fill_betweenx(ypad, np.r_[yl[0], yl, yl[-1]], np.r_[yr[0], yr, yr[-1]],
                                 color=cols[i], label=lab if (j == 0 and idx == 0) else None)
                
                # Add dots at Silt peak if requested and this is the Silt class
                if add_dots and lab == "Silt":
                    yc = (yl + yr) / 2  # Peak of Silt class
                    ax.scatter(yc, ys, color='black', s=25, zorder=10, 
                              edgecolors='white', linewidths=0.75)
        
        # Configure axis
        ax.set(xlim=(0, 100), ylim=(global_max, global_min), title=core_name)
        ax.set_xlabel("Grain Size (%)")
        if idx == 0:
            ax.set_ylabel("Core Depth (mm)")
    
    # Add legend below the plots
    fig.legend(labels, loc="lower center", bbox_to_anchor=(0.5, -0.02), 
               ncol=len(labels), frameon=False)
    fig.suptitle(title, y=1.00, fontsize=16)
    fig.tight_layout()
    fig.savefig(out_pdf, bbox_inches="tight", dpi=300)
    plt.close(fig)
    print(f"✅ Multi-core plot saved as {out_pdf}")

In [19]:
#Cell 5c - Multi-core plotting function with unified y-axis
def plot_multiple_cores(cores_data: dict, bins, labels, title, fig_w_per_core, fig_h, 
                        out_pdf, gap_mm=20, colors=None, add_dots=False):
    """
    Plot multiple cores side-by-side with unified y-axis scale.
    
    Parameters:
    cores_data: dict of {core_name: dataframe}
    add_dots: if True, add dots at measurement points at the Silt/Sand boundary
    """
    n_cores = len(cores_data)
    cols = colors or ["#653700", "#808080", "#E6DAA6"]  # Clay, Silt, Sand
    
    # Find global min/max depth across all cores for unified scale
    all_depths = []
    for data in cores_data.values():
        all_depths.extend(data["Center Depth"].unique())
    global_min = min(all_depths)
    global_max = max(all_depths)
    
    # Create subplots
    fig, axes = plt.subplots(1, n_cores, figsize=(fig_w_per_core * n_cores, fig_h), 
                             sharey=True, squeeze=False)
    axes = axes.flatten()
    
    # Plot each core
    for idx, (core_name, data) in enumerate(sorted(cores_data.items())):
        ax = axes[idx]
        depths = np.sort(data["Center Depth"].unique())
        
        # Process data - use Histogram (%) q3
        binned = pd.cut(data["um"], bins=bins, labels=labels, right=False, include_lowest=True, ordered=True)
        tmp = data.assign(class_=binned).groupby(["Center Depth", "class_"], observed=True)["Histogram (%) q3"].sum().reset_index()
        wide = tmp.pivot(index="Center Depth", columns="class_", values="Histogram (%) q3").reindex(labels, axis=1).fillna(0)
        cumu = wide.cumsum(axis=1).to_numpy()
        
        # Find gaps
        depth_idx = np.arange(len(depths))
        gaps = np.where(np.diff(depths) > gap_mm)[0]
        segs = np.split(depth_idx, gaps + 1)
        
        # Plot each grain size class
        for i, lab in enumerate(labels):
            left = np.zeros(len(depths)) if i == 0 else cumu[:, i-1]
            right = cumu[:, i]
            
            for j, s in enumerate(segs):
                ys = depths[s]
                yl = left[s]
                yr = right[s]
                ypad = np.r_[ys[0]-1e-6, ys, ys[-1]+1e-6]
                ax.fill_betweenx(ypad, np.r_[yl[0], yl, yl[-1]], np.r_[yr[0], yr, yr[-1]],
                                 color=cols[i], label=lab if (j == 0 and idx == 0) else None)
        
        # Add dots at Silt/Sand boundary (right edge of Silt = left edge of Sand)
        if add_dots:
            # Find Silt column index (should be index 1: Clay=0, Silt=1, Sand=2)
            silt_idx = labels.index("Silt")
            silt_sand_boundary = cumu[:, silt_idx]  # Right edge of Silt = cumulative through Silt
            
            # Plot dots for each segment
            for s in segs:
                ys = depths[s]
                boundary_x = silt_sand_boundary[s]
                ax.scatter(boundary_x, ys, color='black', s=25, zorder=10, 
                          edgecolors='white', linewidths=0.75)
        
        # Extract short name (e.g., "KC17" from "NBP2202_KC17_plots")
        # Look for KC or MC followed by numbers
        import re
        match = re.search(r'(KC|MC)\d+', core_name)
        short_name = match.group(0) if match else core_name
        
        # Configure axis
        ax.set(xlim=(0, 100), ylim=(global_max, global_min), title=short_name)
        ax.set_xlabel("Grain Size (%)")
        if idx == 0:
            ax.set_ylabel("Core Depth (mm)")
    
    # Add legend below the plots
    fig.legend(labels, loc="lower center", bbox_to_anchor=(0.5, -0.02), 
               ncol=len(labels), frameon=False)
    fig.suptitle(title, y=1.00, fontsize=16)
    fig.tight_layout()
    fig.savefig(out_pdf, bbox_inches="tight", dpi=300)
    plt.close(fig)
    print(f"✅ Multi-core plot saved as {out_pdf}")

In [20]:
#Cell 6 - Load data from all cores
# Find all cores in the directory
cores = find_cores(directory)
print(f"Found {len(cores)} core(s): {', '.join(cores.keys())}\n")

# Load data for each core
cores_data = {}
for core_name, core_path in cores.items():
    print(f"Loading {core_name}...")
    cores_data[core_name] = read_multiple_data(core_path)
    print(f"  ✓ {len(cores_data[core_name]['Center Depth'].unique())} depth levels")

print("\n✅ All data loaded!")


Found 9 core(s): NBP2202_KC17_plots, NBP2202_KC21_plots, NBP2202_KC22_plots, NBP2202_KC23_graphs, NBP2202_KC24_plots, NBP2202_KC25_plots, NBP2202_KC26_graphs, NBP2202_MC19_plots, NBP2202_MC27_plots

Loading NBP2202_KC17_plots...
  ✓ 17 depth levels
Loading NBP2202_KC21_plots...
  ✓ 12 depth levels
Loading NBP2202_KC22_plots...
  ✓ 20 depth levels
Loading NBP2202_KC23_graphs...
  ✓ 5 depth levels
Loading NBP2202_KC24_plots...
  ✓ 6 depth levels
Loading NBP2202_KC25_plots...
  ✓ 5 depth levels
Loading NBP2202_KC26_graphs...
  ✓ 6 depth levels
Loading NBP2202_MC19_plots...
  ✓ 21 depth levels
Loading NBP2202_MC27_plots...
  ✓ 8 depth levels

✅ All data loaded!


In [21]:
#Cell 7 - Create the multi-core plot
out_path = os.path.join(output_dir, f"wentworth_multicore_{date.today().strftime('%Y%m%d')}.pdf")

# Set add_dots=True to show measurement points, False to hide them
plot_multiple_cores(
    cores_data, 
    wentworth_bins, 
    wentworth_labels, 
    plot_title, 
    fig_w_per_core, 
    fig_h,
    out_path,
    gap_mm=20, 
    colors=["#653700", "#808080", "#E6DAA6"],  # Clay, Silt, Sand
    add_dots=True  # Changed from False to True
)

✅ Multi-core plot saved as D:\Vent\grain size\processed\wentworth_multicore_20251105.pdf
